In [1]:
import sys
sys.path.append('../src')
# from dataloader import create_dataloaders
from dataloader_ER import create_dataloaders,create_dataloaders_multi_label
# from lora_model import LORAEngine,LORAEngineDeberta,LORAEngineDebertaMultiClass
from lora_model_vmap import LORAEngineDebertaMultiClass,LORAEngineDebertaMultiLabel

# from influence import IFEngine
# from influence_hyperinf import IFEngine
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
# model_name_or_path="../../model/deberta-v3-base"
model_name_or_path="../../model/ModernBERT-base/"
# model_name_or_path="../../model/Qwen2.5-0.5B-Instruct/"
# model_name_or_path="../../model/roberta-large"
# task="mrpc"
task = "Router"
noise_ratio=0
batch_size=16
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
# target_modules = 
target_modules = ['Wqkv', 'Wo', 'Wi']
# target_modules=["value_proj"] ## deberta, for roberta, use ["value"]
# target_modules=["q_proj"] ## deberta, for roberta, use ["value"]

device="cuda:7"
num_epochs=3
lr=3e-4


In [3]:

dataloader_outputs = create_dataloaders_multi_label(model_name_or_path=model_name_or_path,
                                        #    task=task,
                                        #    noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                           train_file = '../train/router/train_guided.csv',
                                           valid_file = '../train/router/valid_guided.csv',
                                           test_file = '../train/router/test_guided.csv',
                                           max_length=2048)
# train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs
train_dataloader, eval_dataloader,test_dataloader, tokenized_datasets, collate_fn, label2id, id2label =dataloader_outputs

num_labels = len(label2id)



Dataset({
    features: ['text', 'labels'],
    num_rows: 3300
})


Map:   0%|          | 0/3300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/68097 [00:00<?, ? examples/s]

In [4]:


lora_engine = LORAEngineDebertaMultiLabel(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/router',
                            valid_each_epoch=True,
                            cal_grad_per_sample = False,
                            num_labels=num_labels,
                            label2id=label2id,
                            id2label=id2label,
                            use_lora = True
                            )

In [5]:
lora_engine.build_LORA_model()
all_logits = lora_engine.train_LORA_model()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at ../../model/ModernBERT-base/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,393,042 || all params: 153,011,748 || trainable%: 2.2175
Total Steps:621


100%|██████████| 207/207 [01:15<00:00,  2.75it/s]


Epoch 1: Training Loss = 0.46381768778614374
Epoch 1:Precision: 0.9189, Recall: 0.8293, F1-score: 0.8718


...

Epoch 2: Training Loss = 0.3417798299432377
Epoch 2:Precision: 1.0000, Recall: 0.6897, F1-score: 0.8163


100%|██████████| 207/207 [00:46<00:00,  4.49it/s]


Epoch 3: Training Loss = 0.3256052052197249
Epoch 3:Precision: 0.9231, Recall: 0.9000, F1-score: 0.9114


100%|██████████| 4257/4257 [06:12<00:00, 11.43it/s]


Precision: 0.0668, Recall: 0.8664, F1-score: 0.1241


In [8]:
import torch
torch.save(all_logits,'../train/router/logits.pkl')


In [10]:
torch.save(id2label,'../train/router/id2label.pkl')